# Customer Churn Analysis and Reduction for Subscription Service

In [2]:
import pandas as pd
import plotly.express as px
#import sys
#!{sys.executable} -m pip install plotly kaleido
#!{sys.executable} -m pip install --upgrade nbformat

### Load dataset and perform initial cleaning

In [7]:
DATASET_PATH = "../dataset/clientes.csv"

df = pd.read_csv(DATASET_PATH, encoding="latin1")
df = df.drop("CLIENTNUM", axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Categoria                      10127 non-null  object 
 1   Idade                          10127 non-null  int64  
 2   Sexo                           10127 non-null  object 
 3   Dependentes                    10127 non-null  int64  
 4   Educação                       10127 non-null  object 
 5   Estado Civil                   10127 non-null  object 
 6   Faixa Salarial Anual           10127 non-null  object 
 7   Categoria Cartão               10126 non-null  object 
 8   Meses como Cliente             10127 non-null  int64  
 9   Produtos Contratados           10127 non-null  int64  
 10  Inatividade 12m                10127 non-null  int64  
 11  Contatos 12m                   10127 non-null  int64  
 12  Limite                         10127 non-null 

In [8]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10126 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Categoria                      10126 non-null  object 
 1   Idade                          10126 non-null  int64  
 2   Sexo                           10126 non-null  object 
 3   Dependentes                    10126 non-null  int64  
 4   Educação                       10126 non-null  object 
 5   Estado Civil                   10126 non-null  object 
 6   Faixa Salarial Anual           10126 non-null  object 
 7   Categoria Cartão               10126 non-null  object 
 8   Meses como Cliente             10126 non-null  int64  
 9   Produtos Contratados           10126 non-null  int64  
 10  Inatividade 12m                10126 non-null  int64  
 11  Contatos 12m                   10126 non-null  int64  
 12  Limite                         10126 non-null  floa

In [27]:
df = df.rename(columns={
    'Categoria': 'categoria',
    'Idade': 'idade',
    'Sexo': 'sexo',
    'Dependentes': 'dependentes',
    'Educação': 'nivel_escolaridade',
    'Estado Civil': 'estado_civil',
    'Faixa Salarial Anual': 'faixa_salario_anual',
    'Categoria Cartão': 'categoria_cartao',
    'Meses como Cliente': 'periodo_cliente_meses',
    'Produtos Contratados': 'produtos_contratados',
    'Inatividade 12m': 'inatividade_ano',
    'Limite': 'limite',
    'Limite Consumido': 'limite_consumido',
    'Limite Disponível': 'limite_disponivel',
    'Mudanças Transacoes_Q4_Q1': 'mudancas_transacoes_q4_q1',
    'Valor Transacoes 12m': 'valor_transacoes_ano',
    'Qtde Transacoes 12m': 'qtde_transacoes_ano', 
    'Mudança Qtde Transações_Q4_Q1': 'mudanca_qtde_transacoes_q4_q1',
    'Taxa de Utilização Cartão': 'taxa_cartao'
    })

In [28]:
df.columns

Index(['categoria', 'idade', 'sexo', 'dependentes', 'nivel_escolaridade',
       'estado_civil', 'faixa_salario_anual', 'categoria_cartao',
       'periodo_cliente_meses', 'produtos_contratados', 'inatividade_ano',
       'Contatos 12m', 'limite', 'limite_consumido', 'limite_disponivel',
       'mudancas_transacoes_q4_q1', 'valor_transacoes_ano',
       'qtde_transacoes_ano', 'mudanca_qtde_transacoes_q4_q1', 'taxa_cartao'],
      dtype='object')

### Print churn distribution

In [30]:
print("\nChurn distribution:")
print(df["categoria"].value_counts())
print(df["categoria"].value_counts(normalize=True).map("{:.1%}".format))


Churn distribution:
categoria
Cliente      8499
Cancelado    1627
Name: count, dtype: int64
categoria
Cliente      83.9%
Cancelado    16.1%
Name: proportion, dtype: object


### Analyze contract type and filter out monthly contracts

In [5]:
print("\nAverage values by contract duration:")
print(df.groupby("duracao_contrato").mean(numeric_only=True))


Average values by contract duration:
                      idade  tempo_como_cliente  frequencia_uso  \
duracao_contrato                                                  
Annual            41.655233           30.556892       15.469386   
Monthly           41.584300           30.514080       15.520942   
Quarterly         41.671907           30.421727       15.501413   

                  ligacoes_callcenter  dias_atraso  total_gasto  \
duracao_contrato                                                  
Annual                       5.132990    15.178851   542.665759   
Monthly                      4.991530    15.051875   550.831097   
Quarterly                    5.128374    15.166353   542.306705   

                  meses_ultima_interacao  cancelou  
duracao_contrato                                    
Annual                         15.595168  0.972408  
Monthly                        15.509311  1.000000  
Quarterly                      15.555189  0.972379  


Com as informações agrupadas, é possível notar que os clientes do plano Mensal, possuem uma média de cancelamento igual a 1, ou seja, praticamente todos os clientes que utilizam esse plano fizeram o cancelamento do serviço.

Esse já é um ponto importante dentro da nossa análise, pois existe um plano dessa empresa, onde praticamente todos os clientes fazem o cancelamento do serviço.

In [6]:
print("\nRemoving monthly contracts for further analysis...")
df = df[df["duracao_contrato"] != "Monthly"]
print("Churn distribution after removing monthly contracts:")
print(df["cancelou"].value_counts(normalize=True).map("{:.1%}".format))


Removing monthly contracts for further analysis...
Churn distribution after removing monthly contracts:
cancelou
1.0    97.2%
0.0     2.8%
Name: proportion, dtype: object


Porcentagem de cancelamento diminuiu de 56.7% para 46.1%.
Ainda está um valor muito alto, portanto vamos continuar avaliando até um valor aceitável.

In [ ]:
print("\nSubscription type distribution:")
print(df["assinatura"].value_counts(normalize=True).map("{:.1%}".format))

In [ ]:
print("\nAverage values by subscription type:")
print(df.groupby("assinatura").mean(numeric_only=True))

Verifica-se que há praticamente a mesma quantidade em cada uma das assinaturas, ou seja, é praticamente 1/3 em cada assinatura.

### Plota histogramas para cada feature por cancelamento

In [ ]:
for column in df.columns:
    fig = px.histogram(df, x=column, color='cancelou', width=600)
    fig.show()

Os histogramas que chamam a atenção são:

- Dias de atraso: é possível notar que clientes com mais de 20 dias de atraso cancelam suas assinaturas.

- Ligações ao Call Center: é possível notar que clientes com mais de 5 ligações ao call center cancelam suas assinaturas.

- Idade: é possível notar que clientes com idade superior a 50 anos cancelaram suas assinaturas.

- Total gasto: é possível notar que clientes que gastam menos que R$ 500 cancelaram suas assinaturas.

### Apply filters based on main churn factors and print churn rates

In [ ]:
print("\nFiltering customers with up to 5 call center calls...")
df = df[df["ligacoes_callcenter"] < 5]
display(df["cancelou"].value_counts(normalize=True).map("{:.1%}".format))

In [ ]:
print("\nFiltering customers with 10 or fewer overdue days...")
df = df[df["dias_atraso"] <= 10]
display(df["cancelou"].value_counts(normalize=True).map("{:.1%}".format))

In [ ]:
print("\nFiltering customers aged 50 or less...")
df = df[df["idade"] <= 50]
display(df["cancelou"].value_counts(normalize=True).map("{:.1%}".format))

In [ ]:
print("\nFiltering customers who spent at least R$500...")
df = df[df["total_gasto"] >= 500]
display(df["cancelou"].value_counts(normalize=True).map("{:.1%}".format))

Porcentagem de cancelamento: 56.7% -> 46.1% -> 26.4% -> 18.4% -> 12.1% -> 4.8%

### Conclusão

A análise de dados revelou alguns fatores importantes que influenciam o cancelamento de clientes:

- Tipo de contrato: A maioria dos cancelamentos ocorre entre clientes com contrato mensal, indicando que essa modalidade está fortemente associada à inatividade.
- Dias de atraso: Clientes que acumularam mais de 20 dias de atraso são significativamente mais propensos a cancelar suas assinaturas.
- Interações com o Call Center: Clientes que fizeram mais de 5 chamadas ao call center tendem a cancelar, sugerindo uma possível insatisfação com o atendimento ou com o serviço.
- Idade: A taxa de cancelamento é mais alta entre clientes com idade acima de 50 anos.
- Total gasto: Clientes que gastaram menos de R$ 500 também demonstram uma propensão maior ao cancelamento.

Esses insights indicam os principais pontos de atenção para reduzir o cancelamento e reter clientes de forma mais eficiente.